In [106]:
import torch
import torch.nn as nn
import cv2
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np

In [107]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [108]:
def mnist(img):
    img1 = img.copy()
    #_, img = cv2.threshold(img, 150, 255, cv2.THRESH_TRUNC)
    img1 = cv2.bitwise_not(img1)

    kernel = np.ones((7, 7), np.uint8)
    img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
    img1 = cv2.cvtColor(img1, cv2.COLOR_RGB2GRAY)
    img1 = cv2.dilate(img1, kernel)  # two thicken the hand writing
    #closing = cv2.morphologyEx(img,cv2.MORPH_CLOSE,kernel,)
    #_, img = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)

    #img = cv2.GaussianBlur(img, (5,5),0) # no need it i guess
    closing = cv2.morphologyEx(img1, cv2.MORPH_CLOSE, kernel, )
    resized = cv2.resize(closing, (28, 28), interpolation=cv2.INTER_AREA)  # the size of mnist is 28x28

    return resized

In [109]:
def to_tensor(arr: np.ndarray):
    tensor_arr = torch.from_numpy(arr).float()
    norm_tensor = F.normalize(tensor_arr)
    norm_tensor = norm_tensor.reshape(-1, 28 * 28)

    return norm_tensor.to(device)

In [110]:
input_size = 784
hidden_size = 500
num_classes = 10
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = NeuralNet(input_size, hidden_size, num_classes).to(device)
model.load_state_dict(torch.load('./data/MNIST_model.pt'))
model.eval()

img = cv2.imread('./data/two.png')

img_mnist = mnist(img)
img_mnist = to_tensor(img_mnist)

output = model.forward(img_mnist)

prob = F.softmax(output, dim=1)
prob = Variable(prob)

prob = prob.cpu().numpy()  #用GPU的数据训练的模型保存的参数都是gpu形式的，要显示则先要转回cpu，再转回numpy模式
pred = np.argmax(prob)  #选出概率最大的一个
print(pred.item())

2
